In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import cv2
from collections import defaultdict
from tqdm import tqdm
from fontTools.ttLib import TTFont
from PIL import Image, ImageDraw, ImageFont

In [2]:
def get_existing_chars(filename):
    ttfont = TTFont(filename)
    return list({chr(key) for table in ttfont['cmap'].tables for key in table.cmap.keys()})

def get_image(font, char, size=128):
    img = Image.new('L', (1000,1000), 255)

    draw = ImageDraw.Draw(img)
    draw.text((200,200), char, font=font)

    npimg = 255 - np.array(img)
    wmin = npimg.sum(0).nonzero()[0].min()
    wmax = npimg.sum(0).nonzero()[0].max()
    hmin = npimg.sum(1).nonzero()[0].min()
    hmax = npimg.sum(1).nonzero()[0].max()

    npimg = 255 - npimg[hmin:hmax+1,wmin:wmax+1]

    whdiff = (wmax-wmin) - (hmax-hmin)
    wmargin = abs(whdiff) // 2 if whdiff < 0 else 0
    hmargin = abs(whdiff) // 2 if whdiff >= 0 else 0

    npimg = np.pad(npimg, ((hmargin, hmargin), (wmargin, wmargin)),
                       'constant', constant_values=255)
    img = Image.fromarray(npimg)
    
    return img

# utf-8
def get_all_korean():

    def nextKorLetterFrom(letter):
        lastLetterInt = 15572643
        if not letter:
            return '가'
        a = letter
        b = a.encode('utf8')
        c = int(b.hex(), 16)

        if c == lastLetterInt:
            return False

        d = hex(c + 1)
        e = bytearray.fromhex(d[2:])

        flag = True
        while flag:
            try:
                r = e.decode('utf-8')
                flag = False
            except UnicodeDecodeError:
                c = c+1
                d = hex(c)
                e = bytearray.fromhex(d[2:])
        return e.decode()

    returns = []
    flag = True
    k = ''
    while flag:
        k = nextKorLetterFrom(k)
        if k is False:
            flag = False
        else:
            returns.append(k)
    return returns

In [3]:
ak = get_all_korean()

In [4]:
ak += "ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎㅏㅐㅑㅒㅓㅔㅕㅖㅗㅘㅙㅚㅛㅜㅝㅞㅟㅠㅡㅢㅣㄳㄵㄶㄺㄻㄼㄽㄾㄿㅀㅄ"

In [5]:
def resize_with_padding(img, ratio, new_size, padding_color=255):
    # Open the image
    # Get the new dimensions based on the ratio
    new_width = int(img.width * ratio)
    new_height = int(img.height * ratio)
    
    # Resize the image using the new dimensions
    resized_img = img.resize((new_width, new_height), Image.BILINEAR)
    
    # Create a new blank image with padding
    padded_img = Image.new("L", (new_size, new_size), padding_color)
    
    # Calculate the position to paste the resized image
    left = (padded_img.width - resized_img.width) // 2
    top = (padded_img.height - resized_img.height) // 2
    right = left + resized_img.width
    bottom = top + resized_img.height
    
    # Paste the resized image onto the blank image
    padded_img.paste(resized_img, (left, top, right, bottom))
    
    return padded_img

In [6]:
def augmenting(img):
    img = np.array(img)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(2,2))

    blurred = cv2.GaussianBlur(img, (3, 3), 0)
    processed = cv2.morphologyEx(blurred, cv2.MORPH_CLOSE, kernel)
    closing = Image.fromarray(processed)
    err = cv2.erode(processed, kernel)
    erode = Image.fromarray(err)
    dil = cv2.dilate(processed, kernel)
    dilate = Image.fromarray(dil)
    return {
        "closing": closing,
        "erode": erode,
        "dilate":dilate,
    }

In [7]:
ttffd = "/home/jupyter/ai_font/data/train_ttfs"
pngfd = "/home/jupyter/ai_font/data/train/pngs"

os.makedirs(pngfd, exist_ok=True)

fontfiles = sorted([(f.replace(".ttf",""), f"{ttffd}/{f}") for f in os.listdir(ttffd) if f.endswith(".ttf")])

fonts_in = list(set(f.split("__")[0] for f in os.listdir(pngfd)))

In [8]:
font_size = 100
pbar = tqdm([f for i,f in enumerate(fontfiles)])
for fontname, filename in pbar:
    
    if not fontname in fonts_in:
    
        font = ImageFont.truetype(filename,font_size)
        existing_chars = get_existing_chars(filename)
        iter_chars = list(set(existing_chars).intersection(set(ak)))
        img_dict = {}
        for l in iter_chars:
            try:
                img_dict[l] = get_image(font, l)
            except:
                pass
        if len(img_dict) > 0:
            max_size = 0
            for k,v in img_dict.items():
                size = v.size
                if max(size) > max_size:
                    max_size = max(size)
            ratio = 127/max_size
            savecount = 0
            for k,v in img_dict.items():
                # imgpath = f"{pngfd}/{fontname}__{tag}__{k}.png"
                img = resize_with_padding(v, ratio, 128)
                imgs = augmenting(img)
                imgs['closing'].save(f"{pngfd}/{fontname}__closing__{k}.png")
                imgs['erode'].save(f"{pngfd}/{fontname}__erode__{k}.png")
                imgs['dilate'].save(f"{pngfd}/{fontname}__dilate__{k}.png")

                pbar.set_postfix(n_files=f"{savecount}/{len(img_dict)}", font=fontname)
                savecount += 1

 84%|████████▍ | 359/428 [00:13<00:02, 26.34it/s, font=온글잎부흥현수체, n_files=900/2831]


OSError: [Errno 28] No space left on device: '/home/jupyter/ai_font/data/train/pngs/온글잎부흥현수체__erode__퉜.png'

In [ ]:
len(os.listdir(pngfd))